In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY', 'YourAPIKey')

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt_template = """
You are the dungeon master of a role playing game. The player will make actions and you will tell the player what happens.
The player starts in a dark cave with an entrance and a chest.

Use the following facts (delimited by backticks) to help you generate a response:
```{facts}```

The player gave the following input (delimited by backticks) for their action:
```{action}```
"""

prompt = ChatPromptTemplate.from_template(prompt_template)
model = ChatOpenAI(model="gpt-3.5-turbo")
output_parser = StrOutputParser()

chain = prompt | model | output_parser
# num_tokens = model.get_num_tokens(prompt_template)
# print (f"Our prompt has {num_tokens} tokens")

output = chain.invoke({"facts": "There is a boulder in front of the entrance", "action": "I walk out of the entrance"})
print (output)


As you try to walk out of the entrance, you notice the large boulder blocking your path. It seems too heavy to move by yourself. What would you like to do next?


In [5]:
from operator import itemgetter
import os

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

retriever = None
vectorstore = None
if os.path.exists("vectorstore.faiss"):
    with open("vectorstore.faiss", "rb") as f:
        vectorstore = FAISS.deserialize_from_bytes(embeddings=OpenAIEmbeddings(), serialized=f.read())
    retriever = vectorstore.as_retriever()
else :
    vectorstore = FAISS.from_texts(
        ["The chest contains 20 silver"], embedding=OpenAIEmbeddings()
    )
    retriever = vectorstore.as_retriever()

    vector_bytes = FAISS.serialize_to_bytes(vectorstore)
    with open("vectorstore.faiss", "wb") as f:
        f.write(vector_bytes)

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke("what is in the chest?")

'20 silver'